# Integrated Gradients : IG

입력된 설명 변수에 대해 미분 가능한 모델에 적용할 수 있는 Local Explainer이다.

장점
- **IG** 계산을 위해 별도의 학습데이터 불필요
- *pertubation*을 위한 별도의 모델이나 데이터 셋 불필요

단점
- **baseline**의 선택에 따라 설명 변수의 기여도 상이

<div style="border: 2px solid rgb(233, 224, 222); padding: 10px; background-color: rgb(233, 224, 222); margin: 10px 0;">
    <strong style="font-weight: bold;">● baseline</strong><br>
    예측 중립적인 값으로 예측치에 영향을 미치치 않는 값을 의미한다.<br>
    baseline에 대한 예시는 선형회귀모델에서 0값, 다중분류 task에서 모든 label의 확률을 동일하게 만드는 입력 등이 있다.<br><br>
    <strong style="font-weight: bold;">일반적인 baseline 선택의 예시</strong><br>
    - <strong>tabular</strong>: 각 설명 변수의 평균 혹은 중위수<br>
    - <strong>text</strong>: &ltunknown&gt<br>
    - <strong>image</strong>: 배경색<br>
</div>

<div style="border: 2px solid rgb(233, 224, 222); padding: 10px; background-color: rgb(233, 224, 222); margin: 10px 0;">
    <strong style="font-weight: bold;">● pertubation(교란)</strong><br>
    입력 데이터나 모델에 작은 변경을 가해 이에 대한 반응을 관찰하는 것<br>
    LIME, SHAP 등에 적용된다.
</div>

---

$$
f(\bm{x_i})=\beta_{0}+\beta_{1}x_{1}+\cdots+\beta_{j}x_{j}+\cdots+\beta_{p}x_{p}
$$

선형회귀모델에서 $i$번째 표본에 대한 $j$번째 설명 변수 $x_{j}$가 예측치에 미치는 영향은
$
\frac{\partial{f(\bm{x_i})}}{\partial{x_{ji}}} = \beta_j
$
이므로
$
\beta_{j} x_{ji}
$
가 된다. **baseline**($x_{0j}$) 대비 $x_{ji}$의 기여도(contribution)는 $\beta_{j}(x_{ji}-x_{0j})$이다.


**IG**가 제안된 배경은 비선형 함수에서 위와 같이 gradient와 입력값의 곱으로 계산된 기여도가 왜곡된 결과를 초래하기 때문이다. 다음 단순화된 모델을 예시로 설명한다.

$$
f(x) =
\begin{cases}
    x & ,\text{if } x < 0.8 \\
    0.8 & ,\text{otherwise } 
\end{cases}
$$


<div style="text-align: center;">
  <img src="https://www.tensorflow.org/static/tutorials/interpretability/integrated_gradients_files/output_kMdAKooulVRE_0.png?hl=ko" alt="이미지 설명" />
  <p style="font-size: 12px; color: grey; text-align: center;">
    <em>
        <a href="https://www.tensorflow.org/tutorials/interpretability/integrated_gradients?hl=ko">
            출처: TensorFlow Tutorials IG
        </a>
    </em>
  </p>
</div>

이진분류 task로 좌측 그림은 입력값별 모델의 출력을 시각화하고 있다. 입력값 $x$는 pixel value로 표현되었다. baseline을 0으로 상정하고 앞선 방식으로 x의 기여도를 0.6과 0.8에 대해 계산하면 아래와 같이 0.6의 기여도가 더 큰 것을 알 수 있다. 그러나, 입력값이 0.8인 경우가 모델의 양성 예측 확률을 80%로 만들기 때문에, 0.8인 경우의 기여도가 더 높다. 실제로 높은 기여도를 가진 값이 더 낮은 기여도를 갖고 있다고 왜곡하는 결과를 도출한 것을 확인할 수 있다.
$$
\frac{\partial{f(x)}}{\partial{x}}\times(x-x_0) =
\begin{cases}
    0.6 & ,\text{if } x = 0.6, \\
    0 & ,\text{if } x = 0.8
\end{cases}\\
\text{where } x_0 = 0\\
$$


IG는 해당 문제를 해결하기 위해 특정 값에서의 gradient 대신 baseline과 입력값 사이 구간의 gradient의 평균을 사용한다.

---

## IG의 정의 및 추정

### Path Integration

IG를 이해하기 위해 먼저 *path integration*에 대해 알아야 한다.

$$
\begin{align*}
\int_{C}f(\bm{x})ds
&=\int_{t_0}^{t_1}f(\bm{r}(t))\|\bm{r}'(t)\|dt\\
&=\int_{t_0}^{t_1}f(x_1(t), x_2(t), x_3(t),...)\left\|\frac{\partial{\bm{r}}}{\partial t}\right\|dt
\end{align*}
$$

곡선 $C$에 대하여 스칼라 함수 $f(\bm{x})$를 적분한다.<br>
$C$는 $t$를 이용하여 매개변수화 되어 있으며 $\bm{r}(t)$로 표현된다.<br>
$\bm{r}(t) = [x_1(t), x_2(t), x_3(t), ...]$<br>
$\|r'(t)\|$는 Jacobian이다.

위치에 따른 특정 물리량($f$)의 누적합을 계산한다고 이해할 수 있다.<br>
함수에 대입없이 Jacobian만 사용하여 경로 적분하면, 아래의 그림처럼 곡선의 시작점과 끝점사이의 최단경로를 구한다고 이해할 수 있다.
<div style="text-align: center;">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Path_integration_diagram.svg/390px-Path_integration_diagram.svg.png" alt="이미지 설명" />
  <p style="font-size: 12px; color: grey; text-align: center;">
    <em>
        <a href="https://en.wikipedia.org/wiki/Path_integration">
            출처: Wikipedia- Path integration
        </a>
    </em>
  </p>
</div>

#### Completeness Property

Path Integration은 Completeness 성질을 만족하며, 초기상태와 경로에 따른 변화량의 합은 최종 상태를 결정한다는 것을 의미한다.
$$
f(\bm{r}(t_1))=f(\bm{r}(t_0))+\int_{t_0}^{t_1}f(\bm{r}(t))\|\bm{r}'(t)\|dt\\
$$

### IG의 정의

IG의 정의는 다음과 같다.<br>
$x_{0j}$ 는 $j$번째 설명변수의 baseline값, $f$는 예측 모델,
$\frac{\partial{f(x_{j}^{'})}}{\partial{x_{j}}}$
는 
$\frac{\partial{f(x_{j})}}{\partial{x_{j}}}|_{x_{j}=x_{j}^{'}}$ 를 의미한다.

$$
IG(x_j) = (x_{j} - x_{0j}) \int_{0}^{1} \frac {\partial{f(x_{0j} + \alpha (x_{j} - x_{0j}) )}} {\partial{x_{j}}} d\alpha \\
$$



Path Integration의 정의에서 Jacobian만 사용되었고, 매개변수 $\alpha$로 표현된 경로 $x_{0j} + \alpha (x_{j} - x_{0j})$의 길이이다.<br>
특정 구간(경로)에서의 설명 변수 $x_j$의 기여도로 이해할 수 있다.<br>
여기서 구간은 baseline과 x_j로 구성되므로 baseline 대비 기여도이다.<br>
선형 회귀 모델을 $f$에 대입한 후 $\alpha (x_{j} - x_{0j})$를 $w_j$로 치환하여 계산하면 앞선 기여도가 도출된다.

$$
\begin{align*}
IG(x_j) &= 
(x_j - x_{0j})\int_{0}^{1}\frac {\partial{f(x_{0j} + \alpha (x_{j} - x_{0j}) )}} {\partial{x_{j}}} d\alpha \\
&= (x_j - x_{0j}) \int_{x_{0j}}^{x_j} \frac{\partial f(w_j)}{\partial w_j}
\left| \frac{d\alpha}{dw_j} \right|dw_j \\
&= (x_j - x_{0j}) \int_{x_{0j}}^{x_j} \frac{\partial f(w_j)}{\partial w_j}
(x_j-x_{0j})^{-1}dw_j \\
&= \int_{x_{0j}}^{x_j} \frac{\partial f(w_j)}{\partial w_j}dw_j
= \int_{x_{0j}}^{x_j} \beta_j dw_j
= (x_j-x_{0j})\beta_j

\end{align*}


$$

Path Integration의 Completeness 성질에 의해 다음이 성립된다.

$$
\sum_{j=1}^{k}IG(x_j)=f(\bm{x})-f(\bm{x}_0)
$$

### IG의 추정

IG는 다음과 같이 Riemann 적분의 근사치로 추정할 수 있다.

$$
\hat{IG}(x_j) \approx (x_j - x_{0j})\sum_{k=1}^{m}
\frac {\partial{f(x_{0j} + \frac{k}{m} (x_{j} - x_{0j}) )}} {\partial{x_{j}}}
\frac{1}{m}
$$

이에 따라 모델 $f$의 미분의 평균값으로 해석할 수 있다.

## IG의 적용